<a href="https://colab.research.google.com/github/qri0usmind/crop_price_predictor/blob/main/daily_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

working daily -

In [ ]:
# === Section 1: Imports ===
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# === Section 2: Load + Preprocess Dataset ===

# Load base dataset
df = pd.read_csv("dataset_2.csv")
df['date'] = pd.to_datetime(df['date'])

# Load and parse new rice price row from RIC_IDX.csv
ric_df = pd.read_csv("RIC_IDX.csv")
ric_df = ric_df.rename(columns={
    '取引日': 'date',
    '帳入値': 'contract_price',
    '出来高': 'volume',
    '取組高': 'open_interest'
})
ric_df['date'] = pd.to_datetime(ric_df['date'], format='%Y%m%d')

# Get max volume row from RIC file
ric_max = ric_df.loc[ric_df['volume'].idxmax()].copy()
new_date = ric_max['date']

# Check if already exists
if new_date not in df['date'].values:
    new_row = {
        'date': new_date,
        'rice_price_per_kg': ric_max['contract_price'] / 60,
        'volume': ric_max['volume'],
        'open_interest': ric_max['open_interest'],
        'days_since_last_trade': (new_date - df['date'].max()).days
    }

    # Assign season label
    month, day = new_date.month, new_date.day
    if (month == 3 or month == 4 or month == 5) or (month == 6 and day <= 15):
        season = 'planting'
    elif (month == 6 and day > 15) or (month in [7, 8]) or (month == 9 and day <= 19):
        season = 'growing'
    elif (month == 9 and day >= 20) or month == 10:
        season = 'harvest'
    else:
        season = 'post_harvest'

    new_row['season'] = season
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# Sort by date and reset index
df.sort_values('date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Add lag and moving average
df['rice_price_lag_1'] = df['rice_price_per_kg'].shift(1)
df['rice_price_ma_7'] = df['rice_price_per_kg'].rolling(window=7).mean()

# Save for inspection
df.to_csv("updated_dataset.csv", index=False)


latest = df.iloc[-1:].copy()


weather_url = "https://api.open-meteo.com/v1/forecast?latitude=37.9226&longitude=139.0412&daily=temperature_2m_max,temperature_2m_min,precipitation_sum&models=jma_seamless&timezone=Asia%2FTokyo"
response = requests.get(weather_url)
data = response.json()
daily_data = pd.DataFrame(data['daily'])
daily_data['time'] = pd.to_datetime(daily_data['time'])
tomorrow = (latest['date'].values[0] + np.timedelta64(1, 'D')).astype('datetime64[D]')
tomorrow_row = daily_data[daily_data['time'] == pd.to_datetime(tomorrow)]


if not tomorrow_row.empty:
  temp_min = tomorrow_row['temperature_2m_min'].values[0]
  temp_max = tomorrow_row['temperature_2m_max'].values[0]
  temp_avg = (temp_min + temp_max) / 2
  precipitation = tomorrow_row['precipitation_sum'].values[0]
else:
  temp_avg = np.nan
  precipitation = np.nan


print("Forecast variables loaded:")
print(f"Temp Avg: {temp_avg:.2f} °C")
print(f"Precipitation: {precipitation:.2f} mm")

# Add weather data to df
df['temp_avg'] = temp_avg
df['precipitation'] = precipitation


# === Section 3: Train Forecasting Model (Ridge) ===
df = pd.get_dummies(df, columns=['season'], drop_first=False)
df['target'] = df['rice_price_per_kg'].shift(-1)
df_new = df
df = df.dropna()

X = df.drop(columns=['date', 'rice_price_per_kg', 'target'])
y = df['target']

split_idx = int(len(X) * 0.9)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

ridge = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0])
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

# Evaluate model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n📊 Ridge Model Evaluation:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.4f}")


latest_input = X.iloc[[-1]]
predicted_price = ridge.predict(latest_input)[0]
print(f"\n📈 Predicted rice price for tomorrow: ¥{round(predicted_price, 2)} per kg")

df = df_new
df.to_csv("dataset_2.csv", index=False)

Forecast variables loaded:
Temp Avg: 23.95 °C
Precipitation: 0.10 mm

📊 Ridge Model Evaluation:
MAE: 10.54
MSE: 260.24
R² Score: 0.7646

📈 Predicted rice price for tomorrow: ¥544.54 per kg


In [ ]:
df_new.tail()

,date,rice_price_per_kg,rice_price_lag_1,rice_price_ma_7,volume,open_interest,precipitation,temp_avg,days_since_last_trade,season_growing,season_harvest,season_planting,season_post-harvest,target,season_harvest
1151,2025-09-17,555.833333,558.333333,539.809524,83,23,0.1,23.95,1,True,False,False,False,556.666667,False
1152,2025-09-18,556.666667,555.833333,544.095238,91,40,0.1,23.95,1,True,False,False,False,543.333333,False
1153,2025-09-22,543.333333,556.666667,547.666667,58,70,0.1,23.95,4,False,True,False,False,545.000000,False
1154,2025-09-24,545.000000,543.333333,549.904762,92,120,0.1,23.95,2,False,True,False,False,543.000000,False
1155,2025-09-25,543.000000,545.000000,550.333333,122,112,0.1,23.95,1,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
# === Section 1: Imports ===
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# === Section 2: Load + Preprocess Dataset ===

# Load base dataset
df = pd.read_csv("dataset.csv")
df['date'] = pd.to_datetime(df['date'])

# Load and parse new rice price row from RIC_IDX.csv
ric_df = pd.read_csv("RIC_IDX.csv")
ric_df = ric_df.rename(columns={
    '取引日': 'date',
    '帳入値': 'contract_price',
    '出来高': 'volume',
    '取組高': 'open_interest'
})
ric_df['date'] = pd.to_datetime(ric_df['date'], format='%Y%m%d')

# Get max volume row from RIC file
ric_max = ric_df.loc[ric_df['volume'].idxmax()].copy()
new_date = ric_max['date']

# Check if already exists
if new_date not in df['date'].values:
    new_row = {
        'date': new_date,
        'rice_price_per_kg': ric_max['contract_price'] / 60,
        'volume': ric_max['volume'],
        'open_interest': ric_max['open_interest'],
        'days_since_last_trade': (new_date - df['date'].max()).days
    }

    # Assign season label
    month, day = new_date.month, new_date.day
    if (month == 3 or month == 4 or month == 5) or (month == 6 and day <= 15):
        season = 'planting'
    elif (month == 6 and day > 15) or (month in [7, 8]) or (month == 9 and day <= 19):
        season = 'growing'
    elif (month == 9 and day >= 20) or month == 10:
        season = 'harvest'
    else:
        season = 'post_harvest'

    new_row['season'] = season
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# Sort by date and reset index
df.sort_values('date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Add lag and moving average
df['rice_price_lag_1'] = df['rice_price_per_kg'].shift(1)
df['rice_price_ma_7'] = df['rice_price_per_kg'].rolling(window=7).mean()

# Save for inspection
df.to_csv("updated_dataset.csv", index=False)
print("✅ Section 2 complete: updated_dataset.csv saved.")

# === Section 3: Fetch Weather Forecast & Train Model ===

# Fetch weather forecast for tomorrow
latest = df.iloc[-1:].copy()

weather_url = "https://api.open-meteo.com/v1/forecast?latitude=37.9226&longitude=139.0412&daily=temperature_2m_max,temperature_2m_min,precipitation_sum&models=jma_seamless&timezone=Asia%2FTokyo"
response = requests.get(weather_url)
data = response.json()
daily_data = pd.DataFrame(data['daily'])
daily_data['time'] = pd.to_datetime(daily_data['time'])
tomorrow = (latest['date'].values[0] + np.timedelta64(1, 'D')).astype('datetime64[D]')
tomorrow_row = daily_data[daily_data['time'] == pd.to_datetime(tomorrow)]

if not tomorrow_row.empty:
    temp_min = tomorrow_row['temperature_2m_min'].values[0]
    temp_max = tomorrow_row['temperature_2m_max'].values[0]
    temp_avg = (temp_min + temp_max) / 2
    precipitation = tomorrow_row['precipitation_sum'].values[0]
else:
    temp_avg = np.nan
    precipitation = np.nan

print("\n📦 Forecast variables loaded:")
print(f"Temp Avg: {temp_avg:.2f} °C")
print(f"Precipitation: {precipitation:.2f} mm")

# === Train Forecasting Model & Handle Today's Prediction ===
df = pd.get_dummies(df, columns=['season'], drop_first=False)
df['target'] = df['rice_price_per_kg'].shift(-1)
df_new = df
df = df.dropna()

X = df.drop(columns=['date', 'rice_price_per_kg', 'target'])
y = df['target']

split_idx = int(len(X) * 0.9)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

ridge = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0])
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

# Evaluate model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n📊 Ridge Model Evaluation:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.4f}")

# === 7-Day Rolling Forecast (User Selectable) ===

# Step 1: Build rolling forecast using last known inputs
future_forecasts = []
latest_known_features = X.iloc[-1:].copy()
latest_known_price = df['rice_price_per_kg'].iloc[-1]
last_7_prices = df['rice_price_per_kg'].tail(6).tolist() + [latest_known_price]


for i in range(1, 8):
    forecast_date = df['date'].iloc[-1] + timedelta(days=i)

    # Fetch weather forecast for this date
    tomorrow_row = daily_data[daily_data['time'] == pd.to_datetime(forecast_date)]
    if not tomorrow_row.empty:
        temp_min = tomorrow_row['temperature_2m_min'].values[0]
        temp_max = tomorrow_row['temperature_2m_max'].values[0]
        temp_avg = (temp_min + temp_max) / 2
        precipitation = tomorrow_row['precipitation_sum'].values[0]
    else:
        temp_avg = np.nan
        precipitation = np.nan

    # Estimate features for this date
    new_input_features = latest_known_features.copy()
    new_input_features['rice_price_lag_1'] = latest_known_price
    last_7_prices.append(latest_known_price)
    last_7_prices.pop(0)
    new_input_features['rice_price_ma_7'] = pd.Series(last_7_prices).mean()

    # Assign season
    month, day = pd.to_datetime(forecast_date).month, pd.to_datetime(forecast_date).day
    if (month == 3 or month == 4 or month == 5) or (month == 6 and day <= 15):
        season = 'planting'
    elif (month == 6 and day > 15) or (month in [7, 8]) or (month == 9 and day <= 19):
        season = 'growing'
    elif (month == 9 and day >= 20) or month == 10:
        season = 'harvest'
    else:
        season = 'post_harvest'

    for s in ['season_planting', 'season_growing', 'season_harvest', 'season_post_harvest']:
        new_input_features[f'season_{season}'] = 1 if s.endswith(season) else 0

    # Set weather features
    new_input_features['temp_avg'] = temp_avg
    new_input_features['precipitation'] = precipitation
    new_input_features['days_since_last_trade'] = (forecast_date - df['date'].max()).days

    if x_next.isnull().values.any():
        print(f"⚠️ Skipping {forecast_date.date()} due to missing values in input.")
        continue

    pred_price = ridge.predict(new_input_features)[0]
    future_forecasts.append({'date': pd.to_datetime(forecast_date).date(), 'predicted_price': pred_price})

    # update for next loop
    latest_known_price = pred_price

print("\n📈 7-Day Rolling Forecast:")
for row in future_forecasts:
    print(f"{row['date']}: ¥{round(row['predicted_price'], 2)} per kg")

df = df_new
df.to_csv("dataset_2.csv", index=False)

✅ Section 2 complete: updated_dataset.csv saved.

📦 Forecast variables loaded:
Temp Avg: 21.90 °C
Precipitation: 13.60 mm

📊 Ridge Model Evaluation:
MAE: 10.42
MSE: 258.72
R² Score: 0.7511


NameError: name 'x_next' is not defined

In [ ]:
# === Section 1: Imports ===
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# === Section 2: Load + Preprocess Dataset ===

# Load base dataset
df = pd.read_csv("dataset_2.csv")
df['date'] = pd.to_datetime(df['date'])

# Load and parse new rice price row from RIC_IDX.csv
ric_df = pd.read_csv("RIC_IDX.csv")
ric_df = ric_df.rename(columns={
    '取引日': 'date',
    '帳入値': 'contract_price',
    '出来高': 'volume',
    '取組高': 'open_interest'
})
ric_df['date'] = pd.to_datetime(ric_df['date'], format='%Y%m%d')

# Get max volume row from RIC file
ric_max = ric_df.loc[ric_df['volume'].idxmax()].copy()
new_date = ric_max['date']

# Check if already exists
if new_date not in df['date'].values:
    new_row = {
        'date': new_date,
        'rice_price_per_kg': ric_max['contract_price'] / 60,
        'volume': ric_max['volume'],
        'open_interest': ric_max['open_interest'],
        'days_since_last_trade': (new_date - df['date'].max()).days
    }

    # Assign season label
    month, day = new_date.month, new_date.day
    if (month == 3 or month == 4 or month == 5) or (month == 6 and day <= 15):
        season = 'planting'
    elif (month == 6 and day > 15) or (month in [7, 8]) or (month == 9 and day <= 19):
        season = 'growing'
    elif (month == 9 and day >= 20) or month == 10:
        season = 'harvest'
    else:
        season = 'post_harvest'

    new_row['season'] = season
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# Sort by date and reset index
df.sort_values('date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Add lag and moving average
df['rice_price_lag_1'] = df['rice_price_per_kg'].shift(1)
df['rice_price_ma_7'] = df['rice_price_per_kg'].rolling(window=7).mean()

# Save for inspection
df.to_csv("updated_dataset.csv", index=False)
print("✅ Section 2 complete: updated_dataset.csv saved.")

# === Section 3: Fetch Weather Forecast & Train Model ===

# Fetch weather forecast for tomorrow
latest = df.iloc[-1:].copy()

weather_url = "https://api.open-meteo.com/v1/forecast?latitude=37.9226&longitude=139.0412&daily=temperature_2m_max,temperature_2m_min,precipitation_sum&models=jma_seamless&timezone=Asia%2FTokyo"
response = requests.get(weather_url)
data = response.json()
daily_data = pd.DataFrame(data['daily'])
daily_data['time'] = pd.to_datetime(daily_data['time'])
tomorrow = (latest['date'].values[0] + np.timedelta64(1, 'D')).astype('datetime64[D]')
tomorrow_row = daily_data[daily_data['time'] == pd.to_datetime(tomorrow)]

if not tomorrow_row.empty:
    temp_min = tomorrow_row['temperature_2m_min'].values[0]
    temp_max = tomorrow_row['temperature_2m_max'].values[0]
    temp_avg = (temp_min + temp_max) / 2
    precipitation = tomorrow_row['precipitation_sum'].values[0]
else:
    temp_avg = np.nan
    precipitation = np.nan

print("\n📦 Forecast variables loaded:")
print(f"Temp Avg: {temp_avg:.2f} °C")
print(f"Precipitation: {precipitation:.2f} mm")

df['temp_avg'] = temp_avg
df['precipitation'] = precipitation

# === Train Forecasting Model & Handle Today's Prediction ===
df = pd.get_dummies(df, columns=['season'], drop_first=False)
df['target'] = df['rice_price_per_kg'].shift(-1)
updated_df = df
df = df.dropna()

X = df.drop(columns=['date', 'rice_price_per_kg', 'target'])
y = df['target']

split_idx = int(len(X) * 0.9)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

ridge = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0])
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

# Evaluate model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("📊 Ridge Model Evaluation:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.4f}")

# === 7-Day Rolling Forecast (User Selectable) ===

# Step 1: Build rolling forecast using last known inputs
future_forecasts = []
latest_known = updated_df[updated_df['date'] == updated_df['date'].max()].copy()

for i in range(1, 8):
    forecast_date = latest_known['date'].values[0] + np.timedelta64(i, 'D')

    # Fetch weather forecast for this date
    forecast_date_str = pd.to_datetime(forecast_date).strftime('%Y-%m-%d')
    tomorrow_row = daily_data[daily_data['time'].dt.strftime('%Y-%m-%d') == forecast_date_str]

    if not tomorrow_row.empty:
        temp_min = tomorrow_row['temperature_2m_min'].values[0]
        temp_max = tomorrow_row['temperature_2m_max'].values[0]
        temp_avg = (temp_min + temp_max) / 2
        precipitation = tomorrow_row['precipitation_sum'].values[0]
    else:
        temp_avg = np.nan
        precipitation = np.nan

    # Estimate features for this date
    new_input = latest_known.copy()
    new_input['rice_price_lag_1'] = new_input['rice_price_per_kg']
    new_input['rice_price_per_kg'] = np.nan
    new_input['rice_price_ma_7'] = pd.concat([df['rice_price_per_kg'].tail(6),pd.Series([new_input['rice_price_lag_1'].values[0]])]).rolling(window=7).mean().iloc[-1]
    new_input['date'] = forecast_date
    new_input['days_since_last_trade'] = (forecast_date - df['date'].max()).days
    new_input['temp_avg'] = temp_avg
    new_input['precipitation'] = precipitation

    # Assign season
    month, day = pd.to_datetime(forecast_date).month, pd.to_datetime(forecast_date).day
    if (month == 3 or month == 4 or month == 5) or (month == 6 and day <= 15):
        season = 'planting'
    elif (month == 6 and day > 15) or (month in [7, 8]) or (month == 9 and day <= 19):
        season = 'growing'
    elif (month == 9 and day >= 20) or month == 10:
        season = 'harvest'
    else:
        season = 'post_harvest'

    season_cols = [col for col in df.columns if col.startswith('season_')]
    for s in season_cols:
       new_input[s] = 1 if s.endswith(season) else 0


    x_next = new_input.drop(columns=['date', 'rice_price_per_kg', 'target'])
    if x_next.isnull().values.any():
      print(f"⚠️ Skipping {pd.to_datetime(forecast_date).date()} due to missing values:")
      print(x_next.isnull().sum())  # print which columns are missing
      print(x_next)  # optional: full input row
      continue

    pred_price = ridge.predict(x_next)[0]
    future_forecasts.append({'date': pd.to_datetime(forecast_date).date(), 'predicted_price': pred_price})

    # update for next loop
    latest_known['rice_price_per_kg'] = pred_price
    latest_known['rice_price_ma_7'] = new_input['rice_price_ma_7']

print("📅 7-Day Forecast:")
for row in future_forecasts:
    print(f"{row['date']}: ¥{round(row['predicted_price'], 2)} per kg")
    print(f"📆 Predicted price for next available date ({closest_future_date}): ¥{round(predicted_price, 2)} per kg")

df = df_new
df.to_csv("dataset_2.csv", index=False)


✅ Section 2 complete: updated_dataset.csv saved.

📦 Forecast variables loaded:
Temp Avg: 23.45 °C
Precipitation: 1.50 mm
📊 Ridge Model Evaluation:
MAE: 10.57
MSE: 260.96
R² Score: 0.7566
⚠️ Skipping 2025-10-02 due to missing values:
rice_price_lag_1         0
rice_price_ma_7          0
volume                   0
open_interest            0
precipitation            1
temp_avg                 1
days_since_last_trade    0
season_growing           0
season_harvest           0
season_planting          0
season_post-harvest      0
dtype: int64
      rice_price_lag_1  rice_price_ma_7  volume  open_interest  precipitation  \
1154         555.77672       551.444293     122            112            NaN   

      temp_avg  days_since_last_trade  season_growing  season_harvest  \
1154       NaN                     10               0               1   

      season_planting  season_post-harvest  
1154                0                    1  
📅 7-Day Forecast:
2025-09-26: ¥544.53 per kg


NameError: name 'closest_future_date' is not defined

In [ ]:
forecast_date_str

'2025-10-01'

In [ ]:
tomorrow_row

,time,temperature_2m_max,temperature_2m_min,precipitation_sum
0,2025-09-25,25.0,21.0,10.0


In [ ]:
latest_known

,date,rice_price_per_kg,rice_price_lag_1,rice_price_ma_7,volume,open_interest,precipitation,temp_avg,days_since_last_trade,season_growing,season_harvest,season_planting,season_post-harvest,target
1153,2025-09-24,545.0,556.666667,547.904762,92,120,10.0,23.0,6,False,True,False,False,NaN


In [ ]:
forecast_date

np.datetime64('2025-10-01T00:00:00.000000000')

In [ ]:
new_input

,date,rice_price_per_kg,rice_price_lag_1,rice_price_ma_7,volume,open_interest,precipitation,temp_avg,days_since_last_trade,season_growing,season_harvest,season_planting,season_post-harvest,target
1153,2025-10-01,NaN,545.0,547.904762,92,120,0.4,22.7,13,0,1,0,1,NaN


In [ ]:
df.tail()

,date,rice_price_per_kg,rice_price_lag_1,rice_price_ma_7,volume,open_interest,precipitation,temp_avg,days_since_last_trade,season_growing,season_harvest,season_planting,season_post-harvest,target
1148,2025-09-11,540.000000,529.333333,526.928571,111,101,10.0,23.0,1,True,False,False,False,550.166667
1149,2025-09-12,550.166667,540.000000,530.523810,115,33,10.0,23.0,1,True,False,False,False,558.333333
1150,2025-09-16,558.333333,550.166667,535.285714,100,101,10.0,23.0,4,True,False,False,False,555.833333
1151,2025-09-17,555.833333,558.333333,539.809524,83,23,10.0,23.0,1,True,False,False,False,556.666667
1152,2025-09-18,556.666667,555.833333,544.095238,91,40,10.0,23.0,1,True,False,False,False,545.000000


In [ ]:
updated_df

,date,rice_price_per_kg,rice_price_lag_1,rice_price_ma_7,volume,open_interest,precipitation,temp_avg,days_since_last_trade,season_growing,season_harvest,season_planting,season_post-harvest,target
0,2018-10-16,221.833333,NaN,NaN,50,251,10.0,23.0,0,False,True,False,False,221.833333
1,2018-10-17,221.833333,221.833333,NaN,160,250,10.0,23.0,1,False,True,False,False,221.833333
2,2018-10-18,221.833333,221.833333,NaN,140,250,10.0,23.0,1,False,True,False,False,221.833333
3,2018-10-19,221.833333,221.833333,NaN,100,250,10.0,23.0,1,False,True,False,False,221.833333
4,2018-10-22,221.833333,221.833333,NaN,186,0,10.0,23.0,3,False,True,False,False,226.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,2025-09-12,550.166667,540.000000,530.523810,115,33,10.0,23.0,1,True,False,False,False,558.333333
1150,2025-09-16,558.333333,550.166667,535.285714,100,101,10.0,23.0,4,True,False,False,False,555.833333
1151,2025-09-17,555.833333,558.333333,539.809524,83,23,10.0,23.0,1,True,False,False,False,556.666667
1152,2025-09-18,556.666667,555.833333,544.095238,91,40,10.0,23.0,1,True,False,False,False,545.000000


In [ ]:
# === User Selection: Get Predicted Price for Selected Date ===
from datetime import date

# Step 1: Ask user to input date
user_input = input("\n📥 Enter a date (YYYY-MM-DD) within the next 7 days to get predicted price: ")

try:
    selected_date = pd.to_datetime(user_input).date()

    # Step 2: Search the forecast list
    match = next((row for row in future_forecasts if row['date'] == selected_date), None)

    if match:
        print(f"✅ Predicted rice price on {selected_date}: ¥{round(match['predicted_price'], 2)} per kg")
    else:
        print("❌ Date is outside the forecast range. Please select a date within the next 7 days.")

except Exception as e:
    print("⚠️ Invalid date format. Please enter in YYYY-MM-DD format.")
